In [29]:
[1,2,3] + [4,5,6]

[1, 2, 3, 4, 5, 6]

In [4]:
import numpy as np

lambda_mat = np.full((3,3), 0)
lambda_mat[1,1] = 1
print(lambda_mat)

[[0 0 0]
 [0 1 0]
 [0 0 0]]


In [3]:
from Model.InteractionNode import InteractionNode

In [17]:
print("===")
print("Product={}, PriceLevel={}, Bought={}, #Units={}".format(2,3,"Si",45))


l21 = InteractionNode(3, 4, 0, 0, [])
l22 = InteractionNode(2, 1, 0, 0, [])
l1 = InteractionNode(1, 2, 1, 34, [l21,l22])

l1.printInteractions()

===
Product=2, PriceLevel=3, Bought=Si, #Units=45

USER INTERACTIONS: 
╚═ Product=1, PriceLevel=2, Bought=Yes, #Units=34
  ╚═ Product=3, PriceLevel=4, Bought=No, #Units=0
  ╚═ Product=2, PriceLevel=1, Bought=No, #Units=0


In [19]:
import numpy as np
np.random.uniform(low=0.0, high=1.0, size=None)

0.27985094518957787

In [ ]:
import pprint
import sys
import unittest
from Model.GraphProbabilities import GraphProbabilities
from Model.Product import Product
from Model.UserClass import UserClass
from Model.constants import PROBABILITY_MATRIX, SECONDARY_PRODUCTS

id = 0
conversionRate = [
    [1, 0.3, 0.9, 0.2],
    [1, .2, .56, .12],
    [1, .34, .57, .88],
    [1, .2, .56, .12],
    [1, 0.9, 0.17, 0.96]
]

PROBABILITY_MATRIX_3 = [[0, 1, 1, 0, 0],
                      [0.8, 0, 0, 0.9, 0],
                      [0, 1, 0, 0.7, 0],
                      [0.9, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0]]
clickProbability = GraphProbabilities(PROBABILITY_MATRIX_3)
alphas = [1,0,0,0,0]

n_bought_mean = 2
n_bought_variance = 1
n_user_mean = 10
n_user_variance = 1

productList = [Product(int(key), SECONDARY_PRODUCTS[key]) for key in SECONDARY_PRODUCTS]

Lambda = 1

userClass = UserClass(id=id, conversionRate=conversionRate, clickProbability=clickProbability, alphas=alphas,
                        Lambda=Lambda, n_bought_variance=n_bought_variance, n_bought_mean=n_bought_mean,
                        n_user_mean=n_user_mean, n_user_variance=n_user_variance, productList=productList,
                        features_generator=[{"name":"Over 18","probability":0.6},{"name":"Male","probability":0.9}],
                        debug=False)

# pp = pprint.PrettyPrinter(indent=4)
# pp.pprint(userClass.__dict__)

currentPrice = [0, 0, 0, 0, 0]
userClass.setCurrentPrice(currentPrice)

userInteractions = userClass.generateEpisode()
userInteractions.printInteractions()

In [8]:
click_prob_matrix = [[0, .8, .7, 0, 0],
            [0, 0, 0, .9, .8],
            [0, 1, 0, .75, 0],
            [.9, 0, 0, 0, .82],
            [0, 0, .87, .92, 0]]
click_prob_matrix = np.array(click_prob_matrix).tolist()
print(click_prob_matrix)

[[0.0, 0.8, 0.7, 0.0, 0.0], [0.0, 0.0, 0.0, 0.9, 0.8], [0.0, 1.0, 0.0, 0.75, 0.0], [0.9, 0.0, 0.0, 0.0, 0.82], [0.0, 0.0, 0.87, 0.92, 0.0]]


In [ ]:
def yummy(time, env):
    print("Time: {}".format(time))
    print("Env: {}".format(env.Lambda))
from Environment import Environment
e = Environment(0.3, [userClass])
e.addTimeListener(2, yummy)
e.addTimeListener(2, yummy)
e.round()

In [2]:
from Model.GraphProbabilities import GraphProbabilities
from Model.Product import Product
from Model.UserClass import UserClass
from Model.Evaluator.GraphEvaluator import GraphEvaluator
import numpy as np

CLICK_PROB = [[0, .8, .7, 0, 0],
            [0, 0, 0, .9, .8],
            [0, 1, 0, .75, 0],
            [.9, 0, 0, 0, .82],
            [0, 0, .87, .92, 0]]
L = 0.5
LAMBDA_MATRIX = [[0, 1, L, 0, 0],
                [0, 0, 0, L, 1],
                [0, L, 0, 1, 0],
                [L, 0, 0, 0, 1],
                [0, 0, 1, L, 0]]
conversionRate = [
    [0.9, 0.9, 0.9, 0.9, 0.9],
    [.62, .62, .62, .62, .62],
    [.82, .82, .82, .82, .82],
    [.56, .56, .56, .56, .56],
    [.92, 0.92, 0.92, 0.92, .92]
]
convRates = [.9, .62, .82, .56, .92]
alphas = [0.2, 0.1, 0.3, 0.1, 0.3]
SECONDARY_PRODUCTS = {'0': [1, 2], '1': [4, 3], '2': [3, 1], '3': [4, 0], '4': [2, 3]}
margins = [12,7,9,19,5]

w_matrix = GraphProbabilities(CLICK_PROB)
lambda_matrix = GraphProbabilities(LAMBDA_MATRIX)
conv_matrix = GraphProbabilities(conversionRate)
y_matrix = w_matrix.multiplyElementWise(lambda_matrix)
y_matrix = y_matrix.multiplyElementWise(conv_matrix)

productList = [Product(int(key), SECONDARY_PRODUCTS[key]) for key in SECONDARY_PRODUCTS]
eval = GraphEvaluator(products_list=productList, click_prob_matrix=CLICK_PROB, lambda_prob=0.7, conversion_rates=convRates,
                    alphas=alphas, margins=margins, verbose=True)
print("Expected profit over all interactions: {}".format(eval.computeMargin()))

profitSingle = np.multiply(np.array(alphas),np.multiply(np.array(margins),np.array(convRates))).sum()
print("Expected profit over single products: {}".format(profitSingle))

[[0.0, 0.6480000000000001, 0.19845, 0.0, 0.0], [0.0, 0.0, 0.0, 0.121086, 0.30752], [0.0, 0.23533999999999997, 0.0, 0.5043, 0.0], [0.09878400000000002, 0.0, 0.0, 0.0, 0.25715200000000005], [0.0, 0.0, 0.736368, 0.2725408, 0.0]]
Reached nodes in 1-step: 1; 2; 
Reached nodes in 2-step: 1; 3; 4; 
Reached nodes in 3-step: 2; 3; 4; 
Reached nodes in 4-step: 2; 3; 4; 
Expected value margin for product 0 as starting is 26 

Reached nodes in 1-step: 3; 4; 
Reached nodes in 2-step: 0; 2; 3; 4; 
Reached nodes in 3-step: 0; 2; 3; 
Reached nodes in 4-step: 0; 2; 
Expected value margin for product 1 as starting is 33 

Reached nodes in 1-step: 1; 3; 
Reached nodes in 2-step: 0; 3; 4; 
Reached nodes in 3-step: 0; 1; 3; 4; 
Reached nodes in 4-step: 0; 4; 
Expected value margin for product 2 as starting is 30 

Reached nodes in 1-step: 0; 4; 
Reached nodes in 2-step: 1; 2; 
Reached nodes in 3-step: 1; 4; 
Reached nodes in 4-step: 2; 4; 
Expected value margin for product 3 as starting is 27 

Reached nod

In [1]:
from Model.GraphProbabilities import GraphProbabilities
from Model.Evaluator.GraphEvaluator import GraphEvaluator
from Model.Product import Product
from Model.UserClass import UserClass
import numpy as np

CLICK_PROB = [[0, .8, .7, 0, 0],
            [0, 0, 0, .9, .8],
            [0, 1, 0, .75, 0],
            [.9, 0, 0, 0, .82],
            [0, 0, .87, .92, 0]]
L = 0.5
convRates = [.9, .62, .82, .56, .92]
alphas = [0.2, 0.1, 0.3, 0.1, 0.3]
SECONDARY_PRODUCTS = {'0': [1, 2], '1': [4, 3], '2': [3, 1], '3': [4, 0], '4': [2, 3]}
margins = [12,7,9,19,5]

productList = [Product(int(key), SECONDARY_PRODUCTS[key]) for key in SECONDARY_PRODUCTS]
eval = GraphEvaluator(products_list=productList, click_prob_matrix=CLICK_PROB, lambda_prob=0.7, conversion_rates=convRates,
                    alphas=alphas, margins=margins, verbose=False)
print("Expected profit over all interactions: {}".format(eval.computeMargin()))

profitSingle = np.multiply(np.array(alphas),np.multiply(np.array(margins),np.array(convRates))).sum()
print("Expected profit over single products: {}".format(profitSingle))

Expected profit over all interactions: 30.1
Expected profit over single products: 7.252


In [3]:
from Learner.GreedyLearner import *

conversionRateLevels = [
    [1, 0.8, 0.6, 0.4],
    [1, 0.8, 0.6, 0.4],
    [1, 0.8, 0.6, 0.4],
    [1, 0.8, 0.6, 0.4],
    [1, 0.8, 0.6, 0.4]
]

marginsPerPrice = [[1, 4, 23, 23],  # Product 1
                   [1, 22, 14, 32],  # Product 2
                   [1, 7, 3, 4],  # Product 3
                   [1, 8, 12, 9],  # Product 4
                   [1, 2, 50, 100]  # Product 5
                   ]

gLearner = GreedyLearner()
optimal_arm = []

for i in range(0, 300):
    pulledArm = gLearner.pull_arm()
    margins = []
    convRates = []
    for k in range(0,len(pulledArm)):
        margins.append(marginsPerPrice[k][pulledArm[k]])
        convRates.append(conversionRateLevels[k][pulledArm[k]])

    price_configuration_margin = 0
    eval = GraphEvaluator(products_list=productList, click_prob_matrix=CLICK_PROB, lambda_prob=0.7, conversion_rates=convRates,
                    alphas=alphas, margins=margins, verbose=False)

    overall_margin = eval.computeMargin()
    print("Configuration {}; ConvRates {}; Margins {}; Overall Margin {}".format(pulledArm,convRates,margins,overall_margin))
    gLearner.update(overall_margin)


Configuration [0, 0, 0, 0, 0]; ConvRates [1, 1, 1, 1, 1]; Margins [1, 1, 1, 1, 1]; Overall Margin 4.0
Configuration [1, 0, 0, 0, 0]; ConvRates [0.8, 1, 1, 1, 1]; Margins [4, 1, 1, 1, 1]; Overall Margin 5.6
Configuration [0, 1, 0, 0, 0]; ConvRates [1, 0.8, 1, 1, 1]; Margins [1, 22, 1, 1, 1]; Overall Margin 18.4
Configuration [0, 0, 1, 0, 0]; ConvRates [1, 1, 0.8, 1, 1]; Margins [1, 1, 7, 1, 1]; Overall Margin 6.800000000000001
Configuration [0, 0, 0, 1, 0]; ConvRates [1, 1, 1, 0.8, 1]; Margins [1, 1, 1, 8, 1]; Overall Margin 8.5
Configuration [0, 0, 0, 0, 1]; ConvRates [1, 1, 1, 1, 0.8]; Margins [1, 1, 1, 1, 2]; Overall Margin 4.0
Configuration [1, 1, 0, 0, 0]; ConvRates [0.8, 0.8, 1, 1, 1]; Margins [4, 22, 1, 1, 1]; Overall Margin 20.0
Configuration [0, 2, 0, 0, 0]; ConvRates [1, 0.6, 1, 1, 1]; Margins [1, 14, 1, 1, 1]; Overall Margin 10.3
Configuration [0, 1, 1, 0, 0]; ConvRates [1, 0.8, 0.8, 1, 1]; Margins [1, 22, 7, 1, 1]; Overall Margin 21.8
Configuration [0, 1, 0, 1, 0]; ConvRates

In [1]:
import json
from Environment import Environment
import numpy as np

env = Environment(config_path='./Configs/config1.json')
env.round()[0].printInteractions()



USER INTERACTIONS: Over 18=False, Male=True
┗━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
  ┃ Product=4, PriceLevel=0, Bought=Yes, #Units=2   ┃
  ┃   ■ FirstSlot=1, Bought=1                       ┃
  ┃   ■ SecondSlot=3, Bought=1                      ┃
  ┗━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
    ┗━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
      ┃ Product=1, PriceLevel=0, Bought=Yes, #Units=4   ┃
      ┃   ■ FirstSlot=2, Bought=0                       ┃
      ┃   ■ SecondSlot=4, Bought=0                      ┃
      ┗━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
    ┗━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
      ┃ Product=3, PriceLevel=0, Bought=Yes, #Units=2   ┃
      ┃   ■ FirstSlot=4, Bought=0                       ┃
      ┃   ■ SecondSlot=0, Bought=1                      ┃
      ┗━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
        ┗━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
          ┃ Product=0, PriceLevel=0, Bought

In [2]:
import json
from Environment import Environment
import numpy as np
from Learner.BruteForce import *
from Model.UserClass import *
from Model.Product import *
from Model.GraphProbabilities import *
from Model.Evaluator.GraphEvaluator import GraphEvaluator
from Model.Evaluator.Baseline import Baseline



f = open('./Configs/config1.json')
config = json.load(f)
f.close()
uc = config["classes"][0]

productList = [Product(int(key), uc["secondary"][key]) for key in uc["secondary"]]

conversionRateLevels = uc["conversionRates"]
marginsPerPrice = config["margins"]
click_prob = np.array(uc["clickProbability"])
lambda_p = uc["lambda"]
alphas = uc["alphas"]
num_prices = len(conversionRateLevels[0])
num_prods = len(alphas)

bf = BruteForce(num_prices=num_prices, num_products=num_prods)
optimal_arm = []

for i in range(0, num_prices**num_prods):
    pulledArm = bf.pull_arm()
    margins = []
    convRates = []
    for k in range(0,len(pulledArm)):
        margins.append(marginsPerPrice[k][pulledArm[k]])
        convRates.append(conversionRateLevels[k][pulledArm[k]])

    price_configuration_margin = 0
    eval = GraphEvaluator(products_list=productList, click_prob_matrix=click_prob, lambda_prob=lambda_p, conversion_rates=convRates,
                    alphas=alphas, margins=margins, verbose=False)
    eval2 = Baseline(products_list=productList, click_prob_matrix=click_prob, lambda_prob=lambda_p, conversion_rates=convRates,
                    alphas=alphas, margins=margins, verbose=False)

    overall_margin = eval.computeMargin()
    baseline = eval2.computeMargin()
    print("Configuration {}; ConvRates {}; Margins {}; Overall Margin {}; Baseline {}".format(pulledArm,convRates,margins,int(overall_margin*100)/100,int(baseline*100)/100))
    if overall_margin < baseline:
        print("VAFFANCULOOOOO")
    bf.update(overall_margin)

print(bf.get_optima())

{'lambda': 0.7, 'alphas': [0.3, 0.25, 0.15, 0.15, 0.15], 'conversionRates': [[0.7, 0.7, 0.4, 0.2], [0.9, 0.8, 0.6, 0.2], [0.9, 0.7, 0.5, 0.3], [0.8, 0.7, 0.4, 0.3], [0.9, 0.65, 0.45, 0.2]], 'clickProbability': [[0, 0.5, 0.6, 0, 0], [0, 0, 0.4, 0, 0.2], [0, 0, 0, 0.8, 0.7], [0.6, 0, 0, 0, 0.5], [0, 0.9, 0, 0.3, 0]], 'secondary': {'0': [1, 2], '1': [2, 4], '2': [3, 4], '3': [4, 0], '4': [1, 3]}, 'unitsShape': 2, 'unitsScale': 1, 'usersMean': 12, 'usersVariance': 2, 'features': [{'name': 'Over 18', 'probability': 0.6}, {'name': 'Male', 'probability': 0.9}]}
Configuration [0, 0, 0, 0, 0]; ConvRates [0.7, 0.9, 0.9, 0.8, 0.9]; Margins [1, 10, 12, 3, 8]; Overall Margin 24.15; Baseline 5.52
Configuration [0, 0, 0, 0, 1]; ConvRates [0.7, 0.9, 0.9, 0.8, 0.65]; Margins [1, 10, 12, 3, 10]; Overall Margin 23.45; Baseline 5.41
Configuration [0, 0, 0, 0, 2]; ConvRates [0.7, 0.9, 0.9, 0.8, 0.45]; Margins [1, 10, 12, 3, 17]; Overall Margin 24.3; Baseline 5.58
Configuration [0, 0, 0, 0, 3]; ConvRates [0

In [8]:
f = open('./Configs/config1.json')
config = json.load(f)
f.close()
uc = config["classes"][0]

productList = [Product(int(key), uc["secondary"][key]) for key in uc["secondary"]]

conversionRateLevels = np.array(uc["conversionRates"])
marginsPerPrice = np.array(config["margins"])
res = np.multiply(conversionRateLevels,marginsPerPrice)
print(res)
best_per_prod = []
for i in range(0,res.shape[0]):
    best_per_prod.append(np.max(res[i]))
best_per_prod = np.array(best_per_prod)
print("Best per product: {}".format(best_per_prod))
alphas = np.array(uc["alphas"])
print("Alphas: {}".format(alphas))
print(np.multiply(alphas, best_per_prod))
print(np.multiply(alphas, best_per_prod).sum())

[[ 0.7   1.4   4.    3.2 ]
 [ 9.   16.   21.    8.  ]
 [10.8  10.5   9.    6.3 ]
 [ 2.4   5.6   6.    6.3 ]
 [ 7.2   6.5   7.65  5.2 ]]
Best per product: [ 4.   21.   10.8   6.3   7.65]
Alphas: [0.3  0.25 0.15 0.15 0.15]
[1.2    5.25   1.62   0.945  1.1475]
10.162500000000001


In [10]:
def numberToBase(n, b):
    if n == 0:
        return [0]
    digits = []
    while n:
        digits.append(int(n % b))
        n //= b
    return digits[::-1]

arr = numberToBase(1023,4)
remaining = [0 for i in range(0,5-len(arr))]
print(remaining + arr)

[3, 3, 3, 3, 3]


In [10]:
np.multiply(np.multiply(np.array([0.2, 0.6, 0.9, 0.3, 0.45]),np.array([16, 35, 12, 21, 17])),np.array([0.3, 0.25, 0.15, 0.15, 0.15])).sum()

9.9225

In [10]:
import json
from Environment import Environment
import numpy as np
from Learner.BruteForce import *
from Model.UserClass import *
from Model.Product import *
from Model.GraphProbabilities import *
from Model.Evaluator.GraphEvaluator import GraphEvaluator
from Model.Evaluator.Baseline import Baseline



f = open('./Configs/config1.json')
config = json.load(f)
f.close()
uc = config["classes"][0]

productList = [Product(int(key), uc["secondary"][key]) for key in uc["secondary"]]

conversionRateLevels = uc["conversionRates"]
marginsPerPrice = config["margins"]
click_prob = uc["clickProbability"]
lambda_p = uc["lambda"]
alphas = uc["alphas"]
num_prices = len(conversionRateLevels[0])
num_prods = len(alphas)

optimal_arm = []

pulledArm = [0, 3, 0, 2, 0] # [3, 2, 0, 3, 2] 
margins = []
convRates = []
for k in range(0,len(pulledArm)):
    margins.append(marginsPerPrice[k][pulledArm[k]])
    convRates.append(conversionRateLevels[k][pulledArm[k]])

price_configuration_margin = 0
eval = GraphEvaluator(products_list=productList, click_prob_matrix=click_prob, lambda_prob=lambda_p, conversion_rates=convRates,
                alphas=alphas, margins=margins, verbose=True)
eval2 = Baseline(products_list=productList, click_prob_matrix=click_prob, lambda_prob=lambda_p, conversion_rates=convRates,
                alphas=alphas, margins=margins, verbose=True)

overall_margin = eval.computeMargin()
baseline = eval2.computeMargin()
print("Configuration {}; ConvRates {}; Margins {}; Overall Margin {}; Baseline {}".format(pulledArm,convRates,margins,overall_margin,baseline))



[[0.0, 0.35, 0.294, 0.0, 0.0], [0.0, 0.0, 0.08000000000000002, 0.0, 0.027999999999999997], [0.0, 0.0, 0.0, 0.7200000000000001, 0.44099999999999995], [0.168, 0.0, 0.0, 0.0, 0.2], [0.0, 0.81, 0.0, 0.189, 0.0]]
Reached nodes in 1-step: 1; 2; 
Reached nodes in 2-step: 2; 3; 4; 
Reached nodes in 3-step: 1; 3; 4; 
Reached nodes in 4-step: 1; 3; 4; 
Expected value margin for product 0 as starting is 32 

Reached nodes in 1-step: 2; 4; 
Reached nodes in 2-step: 3; 4; 
Reached nodes in 3-step: 0; 3; 4; 
Reached nodes in 4-step: 0; 2; 
Expected value margin for product 1 as starting is 24 

Reached nodes in 1-step: 3; 4; 
Reached nodes in 2-step: 0; 1; 3; 4; 
Reached nodes in 3-step: 0; 1; 
Reached nodes in 4-step: 1; 4; 
Expected value margin for product 2 as starting is 21 

Reached nodes in 1-step: 0; 4; 
Reached nodes in 2-step: 1; 2; 
Reached nodes in 3-step: 2; 4; 
Reached nodes in 4-step: 1; 4; 
Expected value margin for product 3 as starting is 26 

Reached nodes in 1-step: 1; 3; 
Reache